In [1]:
import os
import sys
from pathlib import Path
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import GithubFileLoader
from langchain_milvus import Milvus
from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from sentence_transformers import SentenceTransformer
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel, ToolCallingAgent
from IPython.display import Image, display
from sandbox_client import SandboxClient

In [2]:
endpoint = os.getenv("PHOENIX_TRACE_ENDPOINT", "http://0.0.0.0:6006/v1/traces")
trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))
SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

In [3]:
ROOT_DIR = Path(os.getcwd()).parent
DATA_DIR = ROOT_DIR / "data"
SCREENSHOT_DIR = DATA_DIR / "screenshots"
print(SCREENSHOT_DIR)
# SCREENSHOT_DIR = ROOT_DIR / "screenshots"

/Users/stijn.hering/Desktop/Master-Thesis/data/screenshots


In [4]:
# Standard HF Model
hf_model = HfApiModel(
    model_id="Qwen/Qwen2.5-Coder-32B-Instruct",
)

In [ ]:
# Use the client with async context manager
async with SandboxClient("ws://localhost:8765", client_id="test") as client:
    print(f"Client ID: {client.client_id}")

    # Test screenshot functionality
    result = await client.get_screenshot_with_metadata()
    print(result)

    if "screenshot_path" in result:
        print("INITIAL MOUSE POSITION")
        screenshot_path = str(SCREENSHOT_DIR / result["screenshot_path"])
        display(Image(filename=screenshot_path))

    # Test code agent
    result = await client.run_pyautogui_code("""
    pyautogui.moveTo(100, 100, duration=1)
    import time
    time.sleep(2)
    pyautogui.hotkey('ctrl', 'alt', 't')
    """)
    print(result)

    if "screenshot_path" in result:
        print("MOVING MOUSE")
        screenshot_path = str(SCREENSHOT_DIR / result["screenshot_path"])
        display(Image(filename=screenshot_path))

2025-02-28 14:59:19 [INFO] Connected to WebSocket server at ws://localhost:8765
2025-02-28 14:59:19 [INFO] Sent: SCREENSHOT


Client ID: test


2025-02-28 14:59:20 [INFO] Sent: EXECUTE_GUI_CODE


{'screenshot_path': 'test/pyautogui-2025-02-28T135920.png', 'mouse_position': [100, 100], 'screen_size': [1024, 768], 'client_id': 'test'}
INITIAL MOUSE POSITION
<IPython.core.display.Image object>


2025-02-28 14:59:23 [INFO] Disconnected from WebSocket server


{'stdout': '', 'stderr': '', 'screenshot': 'test/pyautogui-2025-02-28T135923.png'}
